In [0]:
from pyspark.sql import functions as func
from datetime import datetime
from pyspark.sql import Window


In [0]:
storage_account = 'test1fast'
access_key = 'QnSkjChqVUQWCLs9t+yDSK4w02oQVBjWtP9dOOBhpw1O002GrWnk8LHfsU8Ys16QjNKmjnDw2RbM+AStEQNjww=='
spark.conf.set(
  f"fs.azure.account.key.{storage_account}.blob.core.windows.net",
  f"{access_key}"
)

container_name = 'streaming-events-captured'

try:
    dbutils.fs.mount(
    source=f"wasbs://{container_name}@{storage_account}.blob.core.windows.net",
    mount_point="/mnt/blob_storage",
    extra_configs={
        f"fs.azure.account.key.{storage_account}.blob.core.windows.net": f"{access_key}"
    }
    )
except:
    pass

list_files_to_ingest = dbutils.fs.ls("/mnt/blob_storage")
all_json_files = [some_file.path for some_file in list_files_to_ingest]
reception_time = int(datetime.utcnow().timestamp())
df = spark.read.json(all_json_files)\
                .withColumn("source_file", func.input_file_name())\
                .withColumn("unixReviewTime", func.lit(reception_time)
                            )

max_id = spark.sql('SELECT MAX(ID) AS MAX_ID FROM reviews').toPandas()['MAX_ID'][0]
window_spec = Window.orderBy('partition_number')
df = df.withColumn("ID", func.row_number().over(window_spec) + max_id + 1)

df.select('asin',
 'image',
 'overall',
 'reviewText',
 'reviewerID',
 'reviewerName',
 'style',
 'summary',
 'unixReviewTime',
 'verified',
 'vote',
 'ID').dropDuplicates().write.format("delta").mode("append").insertInto("reviews")